In [1]:
import os
import glob as glob
import pandas as pd
import numpy as np
from events_prep import *

Step 1: Preparing events files
---------------------------------

In [2]:
data_dir = '/home/finc/Dropbox/Projects/SelfReg/data/behavioral/prep/'
out_dir = '/home/finc/Dropbox/Projects/SelfReg/data/behavioral/prep_order/'

task = 'surveyMedley'
files = np.sort(os.listdir(data_dir))

for i in range(len(files)):
    sub = files[i][0:4]
    file_path = f'{data_dir}{sub}_{task}_events.tsv'
    if not os.path.exists(file_path):
        print(f'Survey data not exist for sub-{sub}')
    else:
        file_name = os.path.basename(file_path)
        df = pd.read_csv(file_path, sep = '\t')
        df['trial_type'] = df['item_text'].map(get_items_order()) # remaping item_text to the unique item order
        df['trial_no'] = np.arange(1,41) # adding column storing original order
        df['subject'] = f'sub-{sub}' # adding column with subject ID in BIDS format 
        df = df.drop(['worker_id'], axis=1)  # dropping unncecessary columns
        df.to_csv(f'{out_dir}/sub-{sub}_surveyMedley_events.csv', index = False)
        
df.head()        

Survey data not exist for sub-surv


,movement_onset,onset,response_time,duration,coded_response,experiment_exp_id,item_coding,item_responses,item_text,possible_responses,survey,trial_type,junk,trial_no,subject
0,10.017,1.007,3.722324,3.982,4.0,survey_medley,reverse,"66,89,71,82,77",I have difficulty maintaining my focus on proj...,"66,89,71,82,77",grit,Q06,False,1,sub-s650
1,16.890,12.322,-0.719676,3.982,2.0,survey_medley,reverse,"66,89,71,82,77",New ideas and projects sometimes distract me f...,"66,89,71,82,77",grit,Q01,False,2,sub-s650
2,26.506,21.937,-0.718676,3.982,3.0,survey_medley,reverse,"66,89,71,82,77","I do certain things that are bad for me, if th...","66,89,71,82,77",brief,Q13,False,3,sub-s650
3,36.665,31.143,0.234324,3.982,3.0,survey_medley,forward,"66,89,71,82,77",I am good at resisting temptation.,"66,89,71,82,77",brief,Q09,False,4,sub-s650
4,47.597,40.690,1.619324,3.982,4.0,survey_medley,reverse,"66,89,71,82,77",I often act without thinking through all the a...,"66,89,71,82,77",brief,Q21,False,5,sub-s650


Step 2: Creating and saving tidy data frame with all subject behavioral data
---------------------------------

In [3]:
tidy_dir = '/home/finc/Dropbox/Projects/SelfReg/data/behavioral/'
subs_list = os.listdir(out_dir)
tidy_data = pd.DataFrame()

for sub in subs_list:
    events_prep = pd.read_csv(f'{out_dir}{sub}')
    tidy_data = pd.concat([tidy_data, events_prep], axis = 0)    
    

tidy_data_clean = pd.DataFrame(tidy_data, columns=['subject', 'survey', 'item_coding', 'item_text', 'trial_type', 'coded_response', 'response_time', 'junk'])
for i, item in enumerate(tidy_data_clean.junk):
    if item == True:
         tidy_data_clean.coded_response.iloc[i] = 'NA'

tidy_data_clean.to_csv(f'{tidy_dir}{task}_data_all_subjects_tidy.csv', index = False)


junk = len(np.unique(tidy_data_clean['subject'].values[tidy_data_clean['junk'].values]))
all_subjects = len(pd.unique(tidy_data_clean['subject']))

print(f'Subjects with junk trials: {junk}')
print(f'Subjects without junk trials: {all_subjects - junk}')

/home/finc/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys
/home/finc/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Subjects with junk trials: 49
Subjects without junk trials: 56
